<a href="https://colab.research.google.com/github/Abhishekai1/Email_Assistant/blob/main/agent_email.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# ==================================
# AI Email Agent - Gmail Filter
# ==================================

!pip install beautifulsoup4

import imaplib
import email
from email.header import decode_header
import re
import getpass
import ssl


# 📥 Connect to Gmail via IMAP

def connect_to_email(username, app_password):
    mail = imaplib.IMAP4_SSL("imap.gmail.com")
    mail.login(username, app_password)
    mail.select("inbox")
    return mail


# 📬 Fetch and Parse Email

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def fetch_email_content(mail, email_id):
    res, msg = mail.fetch(email_id, "(RFC822)")
    for response in msg:
        if isinstance(response, tuple):
            msg = email.message_from_bytes(response[1])
            subject, encoding = decode_header(msg["Subject"])[0]
            if isinstance(subject, bytes):
                subject = subject.decode(encoding or "utf-8", errors="ignore")
            body = ""
            if msg.is_multipart():
                for part in msg.walk():
                    if part.get_content_type() == "text/plain":
                        try:
                            body += part.get_payload(decode=True).decode()
                        except:
                            pass
            else:
                try:
                    body = msg.get_payload(decode=True).decode()
                except:
                    pass
            return subject, clean_text(body)
    return "", ""


#  Classify

def classify_email(subject, body):
    text = (subject + " " + body).lower()
    spam_keywords = ["win", "free", "click", "account suspended", "urgent action"]
    important_keywords = ["meeting", "project", "invoice", "update", "deadline"]

    if any(word in text for word in spam_keywords):
        return "spam"
    elif any(word in text for word in important_keywords):
        return "important"
    else:
        return "other"


# 🧹 Take Actions

def move_to_spam(mail, email_id):
    mail.store(email_id, '+X-GM-LABELS', '\\Spam')
    print("Moved to Spam.")

def star_important(mail, email_id):
    mail.store(email_id, '+FLAGS', '\\Flagged')
    print("Starred as Important.")


# 🔁 Main Processing Loop

def process_emails(mail):
    _, messages = mail.search(None, 'UNSEEN')
    email_ids = messages[0].split()

    print(f"\n📨 Found {len(email_ids)} unread emails\n")

    for email_id in email_ids:
        subject, body = fetch_email_content(mail, email_id)
        print(f"Subject: {subject[:80]}")
        label = classify_email(subject, body)
        print(f"➡️ Classified as: {label.upper()}")

        if label == "spam":
            move_to_spam(mail, email_id)
        elif label == "important":
            star_important(mail, email_id)
        print("-" * 50)


# 🔐 Secure Input & Run Agent

# getpass to avoid showing password in plain text
username = input("📧 Enter your Gmail address: ")
app_password = getpass.getpass("🔑 Enter your App Password: ")

try:
    mail = connect_to_email(username, app_password)
    process_emails(mail)
    mail.logout()
    print("✅ Done!")
except Exception as e:
    print("❌ Failed to connect or process:", str(e))


^C


KeyboardInterrupt: 